<a href="https://colab.research.google.com/github/NicoleRichards1998/ray/blob/master/Copy_of_LSTM_easy_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y pyarrow
!pip uninstall -y ray # clean removal of previous install, otherwise version number may cause pip not to upgrade

Found existing installation: pyarrow 6.0.1
Uninstalling pyarrow-6.0.1:
  Successfully uninstalled pyarrow-6.0.1


In [ ]:
!python -m pip install --upgrade pip

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
! ls
%cd gdrive/My Drive/project_folder
! git clone https://github.com/NicoleRichards1998/ray.git
! git pull

Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/project_folder
Cloning into 'ray'...
remote: Enumerating objects: 184883, done.
remote: Total 184883 (delta 0), reused 0 (delta 0), pack-reused 184883
Receiving objects: 100% (184883/184883), 169.15 MiB | 7.63 MiB/s, done.
Resolving deltas: 100% (138277/138277), done.
Checking out files: 100% (4784/4784), done.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [5]:
! git init

Initialized empty Git repository in /content/gdrive/MyDrive/project_folder/.git/


In [6]:
! git pull https://github.com/NicoleRichards1998/ray.git

remote: Enumerating objects: 175130, done.
remote: Total 175130 (delta 0), reused 0 (delta 0), pack-reused 175130
Receiving objects: 100% (175130/175130), 165.35 MiB | 8.82 MiB/s, done.
Resolving deltas: 100% (130968/130968), done.
From https://github.com/NicoleRichards1998/ray
 * branch                HEAD       -> FETCH_HEAD
Checking out files: 100% (4784/4784), done.


In [7]:
!pip install ray

     |████████████████████████████████| 53.2 MB 236 kB/s 
     |████████████████████████████████| 4.1 MB 36.8 MB/s 
     |████████████████████████████████| 144 kB 78.1 MB/s 
     |████████████████████████████████| 8.8 MB 71.4 MB/s 
     |████████████████████████████████| 461 kB 71.4 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [ ]:
!pip install tf-estimator-nightly==2.8.0.dev2021122109

     |████████████████████████████████| 462 kB 29.6 MB/s 


In [ ]:
!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-2.0.0.dev0-cp37-cp37m-manylinux2014_x86_64.whl # minimal install

     |████████████████████████████████| 53.6 MB 1.3 MB/s 
     |████████████████████████████████| 4.1 MB 25.0 MB/s 
     |████████████████████████████████| 8.8 MB 40.8 MB/s 
     |████████████████████████████████| 144 kB 58.0 MB/s 
     |████████████████████████████████| 461 kB 59.0 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0


In [ ]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 KB 10.3 MB/s eta 0:00:00


In [ ]:
!pip install lz4

     |████████████████████████████████| 1.2 MB 24.1 MB/s 


In [ ]:
# A hack to force the runtime to restart, needed to include the above dependencies.
import os
os._exit(0)

In [16]:
%cd /content/gdrive/MyDrive/project_folder/ray

/content/gdrive/MyDrive/project_folder/ray


In [17]:
import sys, os
sys.path.append(os.getcwd())

In [20]:
import ray
#!pip install git+https://github.com/ray-project/ray.git
import functools
import gym
from math import log
import numpy as np
import tree  # pip install dm_tree
from typing import Optional
from ray.rllib.agents.ppo import PPOTrainer
import pandas as pd

import psutil
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total



ModuleNotFoundError: ignored

In [13]:
%cd /content/gdrive/MyDrive/project_folder/ray
! git pull

/content/gdrive/MyDrive/project_folder/ray
Already up to date.


In [2]:
# Define your problem using python and openAI's gym API:
class ParrotEnv(gym.Env):
    """Environment in which an agent must learn to repeat the seen observations.
    Observations are float numbers indicating the to-be-repeated values,
    e.g. -1.0, 5.1, or 3.2.
    The action space is always the same as the observation space.
    Rewards are r=-abs(observation - action), for all steps.
    """

    def __init__(self, config):
        # Make the space (for actions and observations) configurable.
        self.action_space = config.get(
            "parrot_shriek_range", gym.spaces.Box(-1.0, 1.0, shape=(1,))
        )
        # Since actions should repeat observations, their spaces must be the
        # same.
        self.observation_space = self.action_space
        self.cur_obs = None
        self.episode_len = 0

    def reset(self):
        """Resets the episode and returns the initial observation of the new one."""
        # Reset the episode len.
        self.episode_len = 0
        # Sample a random number from our observation space.
        self.cur_obs = self.observation_space.sample()
        # Return initial observation.
        return self.cur_obs

    def step(self, action):
        """Takes a single step in the episode given `action`
        Returns: New observation, reward, done-flag, info-dict (empty).
        """
        # Set `done` flag after 10 steps.
        self.episode_len += 1
        done = self.episode_len >= 10
        # r = -abs(obs - action)

        #print(type(self.cur_obs))
        #print(type(action))
        

        reward = int(-abs(self.cur_obs - action))
        #print(type(reward))
        # Set a new observation (random sample).
        self.cur_obs = self.observation_space.sample()
        return self.cur_obs, reward, done, {}


# Create an RLlib Trainer instance to learn how to act in the above
# environment.
trainer = PPOTrainer(
    config={
        # Env class to use (here: our gym.Env sub-class from above).
        "env": ParrotEnv,
        # Config dict to be passed to our custom env's constructor.
        "env_config": {"parrot_shriek_range": gym.spaces.Box(-5.0, 5.0, (1,))},
        # Parallelize environment rollouts.
        "num_workers": 1,
        "model":{
            "use_lstm": True,
            "lstm_cell_size": 256,
            "lstm_use_prev_action": True,
            "lstm_use_prev_reward": True
        }
    }
)

# Train for n iterations and report results (mean episode rewards).
# Since we have to guess 10 times and the optimal reward is 0.0
# (exact match between observation and action value),
# we can expect to reach an optimal episode reward of 0.0.
for i in range(2):
    results = trainer.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")

# Perform inference (action computations) based on given env observations.
# Note that we are using a slightly simpler env here (-3.0 to 3.0, instead
# of -5.0 to 5.0!), however, this should still work as the agent has
# (hopefully) learned to "just always repeat the observation!".
env = ParrotEnv({"parrot_shriek_range": gym.spaces.Box(-3.0, 3.0, (1,))})
# Get the initial observation (some value between -10.0 and 10.0).
obs = env.reset()
state = [np.zeros([256], np.float32) for _ in range(2)]
prev_a = 0
prev_r = 0.0
done = False
total_reward = 0.0
# Play one episode.
while not done:
    # Compute a single action, given the current observation
    # from the environment.
    action, state, _ = trainer.compute_single_action(obs, state, prev_action=prev_a, prev_reward=prev_r)
    # Apply the computed action in the environment.
    obs, reward, done, info = env.step(action)
    prev_a = action
    prev_r = reward
    # Sum up rewards for reporting purposes.
    total_reward += reward
# Report results.
print(f"Played 1 episode; total-reward={total_reward}")

Could not instantiate TBXLogger: No module named 'tensorboardX'.
(pid=584) 2022-04-26 07:13:39,817	WARNING compression.py:18 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(RolloutWorker pid=584) 2022-04-26 07:13:40,495	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
(RolloutWorker pid=584) 2022-04-26 07:13:40,495	WARNING env.py:121 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your envir

Iter: 0; avg. reward=-32.52


2022-04-26 07:15:11,557	ERROR tf_run_builder.py:52 -- Error fetching: [<tf.Tensor 'default_policy/cond_1/Merge:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'default_policy/model_1/lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'default_policy/model_1/lstm/while/Exit_4:0' shape=(?, 256) dtype=float32>, {'action_prob': <tf.Tensor 'default_policy/Exp_1:0' shape=(?,) dtype=float32>, 'action_logp': <tf.Tensor 'default_policy/cond_2/Merge:0' shape=(?,) dtype=float32>, 'action_dist_inputs': <tf.Tensor 'default_policy/Reshape_3:0' shape=(?, 2) dtype=float32>, 'vf_preds': <tf.Tensor 'default_policy/Reshape_4:0' shape=(?,) dtype=float32>}], feed_dict={<tf.Tensor 'default_policy/obs:0' shape=(?, 1) dtype=float32>: array([[2.817867]], dtype=float32), <tf.Tensor 'default_policy/state_in_0:0' shape=(?, 256) dtype=float32>: array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0

Iter: 1; avg. reward=-26.87


ValueError: ignored

In [ ]:
ray.shutdown()